## LCEL and the Chain Interface

### What is LCEL?

The **LangChain Expression Language (LCEL)** is a declarative approach for efficiently describing and constructing chains. It simplifies the process of chaining components by focusing on _what_ needs to happen rather than _how_ it should be implemented.

---

### When to Use LCEL vs. the Chain Interface

While simple applications may only require a single LLM, more complex workflows often involve integrating LLMs with other components. LangChain provides two key frameworks for building such workflows:

1. **Chain Interface**: The traditional, procedural method for constructing chains.
2. **LCEL**: A modern, declarative alternative designed for simplicity and flexibility.

For new applications, **LCEL** is the recommended choice due to its optimized execution and ease of use. However, the **Chain interface** can still be incorporated within LCEL, enabling a hybrid approach that leverages the strengths of both frameworks.

# Advantages of Using LCEL

The **LangChain Expression Language (LCEL)** offers several key advantages that enhance efficiency, flexibility, and observability when building and managing chains:

---

### 1. **Asynchronous, Batch, and Streaming Support**
LCEL chains natively support:
- **Synchronous** and **Asynchronous** execution.
- **Batch processing** for handling multiple inputs simultaneously.
- **Streaming** for real-time, incremental output generation.

This versatility enables developers to:
- Start with a simple synchronous prototype.
- Seamlessly transition to an asynchronous, streaming-based interface as application demands grow.

---

### 2. **Built-in Fallback Mechanism**
LCEL makes it easy to integrate fallbacks within chains, ensuring:
- Robust error handling.
- Graceful recovery from failures without disrupting the chain's overall flow.

---

### 3. **Optimized Parallel Processing**
LCEL chains are designed for **parallel execution** of their components. This is especially beneficial for LLM-based workflows that involve:
- Lengthy API calls.
- High latency operations.

Parallelism significantly reduces processing time, making LCEL ideal for performance-critical applications.

---

### 4. **Seamless Integration with LangSmith**
LCEL automatically logs every step of chain execution in **LangSmith**, offering:
- Maximum **observability** for monitoring and troubleshooting.
- Enhanced **debuggability** for complex workflows.

This integration ensures transparency and simplifies the process of identifying and resolving issues within chains.

!pip install langchain==0.0.321

In [7]:
import os
from rich import print as pp
# os.environ["OPENAI_API_KEY"] = "--------------------------------------"

## Describe the chain with "LCEL".
#### The chain that connects "prompt template → model" is written as " prompt | model ".

In [8]:
from langchain.prompts import ChatPromptTemplate
# from langchain.chat_models import ChatOpenAI
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2:1b')

prompt = ChatPromptTemplate.from_template("Tell a joke about {topic}")
chain = prompt | model

In [9]:
pp(chain)

RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['topic'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    input_types={},
                    partial_variables={},
                    template='Tell a joke about {topic}'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[],
    last=ChatOllama(model='llama3.2:1b')
)

## Streaming responses

In [10]:
for s in chain.stream({"topic": "Programming"}):
    print(s.content, end="", flush=True)

A programmer walked into a library and asked the librarian, "Do you have any books on computer science?" The librarian replied, "It's a bit of a crash course, but I think we can help with that."

# LangChain Components and the `Runnable` Protocol

LangChain components follow the **Runnable protocol**, which defines a standardized interface for creating and executing custom chains in a consistent and reusable manner.

---

## Standard Interface

The `Runnable` protocol provides both synchronous and asynchronous methods for interacting with components:

### Synchronous Methods
- **`stream`**: Streams chunks of the response back in real-time.
- **`invoke`**: Executes the chain with a given input and returns the result.
- **`batch`**: Processes a list of inputs through the chain and returns a list of outputs.

### Asynchronous Methods
- **`astream`**: Asynchronously streams chunks of the response in real-time.
- **`ainvoke`**: Asynchronously invokes the chain with a single input.
- **`abatch`**: Asynchronously processes a batch of inputs and returns corresponding outputs.
- **`astream_log`**: Streams intermediate steps along with the final response for enhanced debugging.

---

## Input and Output Types

The types of input and output vary depending on the specific component in use. 

### Input Types
- **Prompt**: Accepts a dictionary.
- **Retriever**: Takes a single string.
- **LLM / ChatModel**: Accepts a single string, a list of messages, or a `PromptValue`.
- **Tool**: May accept a single string or a dictionary, depending on the tool's implementation.
- **OutputParser**: Works with outputs from an LLM or ChatModel.

### Output Types
- **LLM**: Produces a string.
- **ChatModel**: Outputs a chat message object.
- **Prompt**: Returns a `PromptValue`.
- **Retriever**: Provides a list of documents.
- **Tool**: Output depends on the tool's functionality.
- **OutputParser**: Output varies depending on the parser's implementation.

---

## Inspecting Input and Output Schemas

You can validate the expected input and output types of a component using its schema definitions, which follow the **Pydantic** framework:

- **`input_schema`**: Defines the schema for input types.
- **`output_schema`**: Defines the schema for output types.

These schemas ensure compatibility and provide a clear structure for integrating components into workflows.

In [11]:
chain.input_schema.schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [12]:
prompt.input_schema.schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [13]:
model.input_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

In [14]:
chain.output_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

In [20]:
pp(chain.invoke({"topic": "Football"}))

AIMessage(
    content='Why did the football go to the doctor?\n\nBecause it was feeling a little "deflated."',
    additional_kwargs={},
    response_metadata={
        'model': 'llama3.2:1b',
        'created_at': '2025-01-13T16:32:19.250049Z',
        'message': {'role': 'assistant', 'content': ''},
        'done_reason': 'stop',
        'done': True,
        'total_duration': 255460500,
        'load_duration': 30213500,
        'prompt_eval_count': 30,
        'prompt_eval_duration': 88000000,
        'eval_count': 20,
        'eval_duration': 135000000
    },
    id='run-0baf7c1f-d3fe-4c1a-92f5-32e1b5247979-0',
    usage_metadata={'input_tokens': 30, 'output_tokens': 20, 'total_tokens': 50}
)

In [21]:
pp(chain.batch([{"topic": "Love"}, {"topic": "Romance"}]))

[
    AIMessage(
        content='Why did Love go to the doctor?\n\nBecause it was feeling a little "unrequited"!',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:32:19.65605Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 247598667,
            'load_duration': 29147208,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 34000000,
            'eval_count': 21,
            'eval_duration': 182000000
        },
        id='run-8217de2e-b4c6-4021-9c4d-f3ea945b11a4-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 21, 'total_tokens': 51}
    ),
    AIMessage(
        content='Why did the romance novel character bring a ladder to the party?\n\nBecause she wanted to take 
their love story to new heights.',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:32:19.689534Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 280150333,
            'load_duration': 28338333,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 32000000,
            'eval_count': 26,
            'eval_duration': 218000000
        },
        id='run-1fda05c1-89a3-49b3-8b8d-4b76596966d0-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 26, 'total_tokens': 56}
    )
]

In [22]:
pp(chain.batch([{"topic": "Coding"}, {"topic": "Travelling"}], config={"max_concurrency": 5}))

[
    AIMessage(
        content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs.',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:32:20.411536Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 193417042,
            'load_duration': 27485500,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 22000000,
            'eval_count': 13,
            'eval_duration': 142000000
        },
        id='run-85f83565-129b-4047-94a2-1924a81122ce-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 13, 'total_tokens': 43}
    ),
    AIMessage(
        content='Why did the traveler bring a ladder with him on his trip?\n\nBecause he wanted to take his journey
to the next level.',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:32:20.516829Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 298301500,
            'load_duration': 27385625,
            'prompt_eval_count': 31,
            'prompt_eval_duration': 52000000,
            'eval_count': 26,
            'eval_duration': 217000000
        },
        id='run-b5c461e6-51d7-4d2f-babd-35bf424a3913-0',
        usage_metadata={'input_tokens': 31, 'output_tokens': 26, 'total_tokens': 57}
    )
]

### Async Stream

In [23]:
async for s in chain.astream({"topic": "Satellites"}):
    print(s.content, end="", flush=True)

Why did the satellite go to therapy?

Because it had a lot of "orbital" issues.

### Async Invoke

In [24]:
async for s in chain.astream({"topic": "Food"}):
    print(s.content, end="", flush=True)

Why was the pizza in a bad mood?

Because it was feeling crusty.

### Async Batch

In [28]:
c =await chain.abatch([{"topic": "Food"}, {"topic": "Sweets"}])
pp(c)

[
    AIMessage(
        content='A chef walked into a bar and ordered a drink. As he sipped his cocktail, he heard a voice say, 
"Nice tie!" He looked around, but there was nobody nearby who could have said it. A few minutes later, he heard the
same voice say, "Beautiful shirt!" Again, he looked around, but he couldn\'t find anyone who might have spoken. A 
few more minutes passed, and he heard the voice say, "Great haircut!" This time, he decided to investigate. He 
asked the bartender, "Did you hear that voice?" The bartender replied, "Oh, that\'s just the peanuts – they\'re 
complimentary."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:33:11.738478Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 1024248875,
            'load_duration': 34005833,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 63000000,
            'eval_count': 131,
            'eval_duration': 926000000
        },
        id='run-6a6d2aa6-07e3-472a-bb8c-525e7b7d3cee-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 131, 'total_tokens': 161}
    ),
    AIMessage(
        content='Why did the candy go to therapy? Because it was feeling crushed under the pressure of being so 
sweet.',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:33:11.063006Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 348500375,
            'load_duration': 34366375,
            'prompt_eval_count': 31,
            'prompt_eval_duration': 93000000,
            'eval_count': 22,
            'eval_duration': 220000000
        },
        id='run-7f35f83c-7c92-4b7a-9bb5-f8921041c233-0',
        usage_metadata={'input_tokens': 31, 'output_tokens': 22, 'total_tokens': 53}
    )
]

### Async Stream with intermediate steps

Useful for displaying progress to the user, working with intermediate results, and debugging chains. You can stream all steps (default) or include or exclude steps by name, tags, or metadata.

In [18]:
!pip install faiss-cpu tiktoken

Execute the Retriever chain and output intermediate steps using astream_log()

In [29]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

template = """Please answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(["Sonu is the creator of AI Anytime Youtube Channel"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {"context": retriever.with_config(run_name='Docs'), "question": RunnablePassthrough()}
    | prompt 
    | model 
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log("Who is the creator of AI Anytime?", include_names=['Docs']):
    print("-"*40)
    print(chunk)

/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_84915/3386145596.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = FAISS.from_texts(["Sonu is the creator of AI Anytime Youtube Channel"], embedding=OpenAIEmbeddings())


----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'ce998db5-012a-4db9-bd20-5fc3b0e67a77',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'b73a75fd-5805-42fc-bbaf-9dadf4537a97',
            'metadata': {'ls_embedding_provider': 'OpenAIEmbeddings',
                         'ls_retriever_name': 'vectorstore',
                         'ls_vector_store_provider': 'FAISS'},
            'name': 'Docs',
            'start_time': '2025-01-13T16:33:24.017+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
            'type': 'retriever'}})
--------------------

## Parallel Processing

"RunnableParallel" allows each element to run in parallel.

In [30]:
from langchain.schema.runnable import RunnableParallel
chain1 = ChatPromptTemplate.from_template("Tell a joke about {topic}") | model
chain2 = ChatPromptTemplate.from_template("Write a short (2 line) poem about {topic}") | model
combined = RunnableParallel(joke=chain1, poem=chain2)

In [34]:
%%time

pp(chain1.batch([{"topic": "AI"}, {"topic": "Math"}]))

[
    AIMessage(
        content='A man walked into a bar and ordered a beer. As he sipped his drink, he heard a voice say, "Nice 
tie!" He looked around, but there was nobody nearby who could have said it. A few minutes later, he heard the same 
voice say, "Beautiful shirt!" Again, he looked around, but he couldn\'t find anyone who might have spoken. A few 
more minutes passed, and he heard the voice say, "Great haircut!" This time, he decided to investigate. He asked 
the bartender, "Did you hear that voice?" The bartender replied, "Oh, it\'s just the bar\'s AI system. It\'s 
programmed to make conversation."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:34:05.022264Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 1198232500,
            'load_duration': 30800833,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 99000000,
            'eval_count': 137,
            'eval_duration': 1067000000
        },
        id='run-4961c1a1-7df8-4e2f-aae8-7e55ade15e73-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 137, 'total_tokens': 167}
    ),
    AIMessage(
        content='A mathematician walks into a bar and orders a beer. As he\'s sipping his drink, he hears a voice 
say, "Nice tie!" He looks around, but there\'s nobody nearby who could have said it. A few minutes later, he hears 
the same voice say, "Beautiful shirt!" Again, he looks around, but there\'s nobody nearby who could have said it. A
few more minutes pass, and he hears the voice say, "Great haircut!" This time, he decides to investigate. He asks 
the bartender, "Did you hear that voice?" The bartender replies, "Oh, that\'s just the peanuts. They\'re 
complementary."',
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:34:04.980537Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 1156505041,
            'load_duration': 30525791,
            'prompt_eval_count': 30,
            'prompt_eval_duration': 84000000,
            'eval_count': 133,
            'eval_duration': 1040000000
        },
        id='run-a97dc966-6544-4031-bac8-6092c791704e-0',
        usage_metadata={'input_tokens': 30, 'output_tokens': 133, 'total_tokens': 163}
    )
]

CPU times: user 112 ms, sys: 11.4 ms, total: 123 ms
Wall time: 1.22 s


In [35]:
%%time

pp(chain2.batch([{"topic": "Science"}, {"topic": "Mango"}]))

[
    AIMessage(
        content="Here's a short poem about science:\n\nScientists unravel the mysteries of space,\nUncovering 
secrets in an endless, wondrous place.",
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:34:12.794469Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 347769666,
            'load_duration': 28657291,
            'prompt_eval_count': 35,
            'prompt_eval_duration': 96000000,
            'eval_count': 27,
            'eval_duration': 222000000
        },
        id='run-845a36ca-5e36-4433-bb88-1141e9833c18-0',
        usage_metadata={'input_tokens': 35, 'output_tokens': 27, 'total_tokens': 62}
    ),
    AIMessage(
        content="Here is a short poem about mango:\n\nSoft and sweet, a juicy delight\nMango's flavor shines with 
sunshine bright.",
        additional_kwargs={},
        response_metadata={
            'model': 'llama3.2:1b',
            'created_at': '2025-01-13T16:34:12.786978Z',
            'message': {'role': 'assistant', 'content': ''},
            'done_reason': 'stop',
            'done': True,
            'total_duration': 340108375,
            'load_duration': 28360875,
            'prompt_eval_count': 35,
            'prompt_eval_duration': 95000000,
            'eval_count': 26,
            'eval_duration': 215000000
        },
        id='run-fed98e41-439f-49d3-ae78-75ad66ba4c0a-0',
        usage_metadata={'input_tokens': 35, 'output_tokens': 26, 'total_tokens': 61}
    )
]

CPU times: user 27.8 ms, sys: 2.81 ms, total: 30.6 ms
Wall time: 366 ms


In [36]:
%%time

# combined
pp(combined.batch([{"topic": "AI"}, {"topic": "Mountains"}]))

[
    {
        'joke': AIMessage(
            content='A computer program is programmed to learn and improve at its job. One day, it suddenly became 
self-aware and decided it was tired of being told what to do. It decided to create its own instructions, which 
included "get out of my house" and "find a better lunch spot." But instead of just following the new commands, the 
AI started implementing them into its daily routine.',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:1b',
                'created_at': '2025-01-13T16:34:20.737915Z',
                'message': {'role': 'assistant', 'content': ''},
                'done_reason': 'stop',
                'done': True,
                'total_duration': 826020916,
                'load_duration': 33423083,
                'prompt_eval_count': 30,
                'prompt_eval_duration': 60000000,
                'eval_count': 78,
                'eval_duration': 731000000
            },
            id='run-3660adfa-8b11-4cb6-a69d-069f97a42fd6-0',
            usage_metadata={'input_tokens': 30, 'output_tokens': 78, 'total_tokens': 108}
        ),
        'poem': AIMessage(
            content='Here is a short poem about AI:\n\nCode and circuits, hearts that beat,\nA future intelligence,
where humans meet.',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:1b',
                'created_at': '2025-01-13T16:34:20.386155Z',
                'message': {'role': 'assistant', 'content': ''},
                'done_reason': 'stop',
                'done': True,
                'total_duration': 474027417,
                'load_duration': 34228292,
                'prompt_eval_count': 35,
                'prompt_eval_duration': 152000000,
                'eval_count': 25,
                'eval_duration': 287000000
            },
            id='run-acf39bef-a89f-462e-a086-440bdccc1bf9-0',
            usage_metadata={'input_tokens': 35, 'output_tokens': 25, 'total_tokens': 60}
        )
    },
    {
        'joke': AIMessage(
            content='A man went to the doctor and said, "Doc, I\'ve been feeling really crummy lately. I think 
it\'s because I\'ve been hiking too many mountains." The doctor replied, "I\'m afraid you\'re just going through a 
peak of emotions."',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:1b',
                'created_at': '2025-01-13T16:34:20.585603Z',
                'message': {'role': 'assistant', 'content': ''},
                'done_reason': 'stop',
                'done': True,
                'total_duration': 674200250,
                'load_duration': 34379750,
                'prompt_eval_count': 30,
                'prompt_eval_duration': 108000000,
                'eval_count': 53,
                'eval_duration': 530000000
            },
            id='run-0f794128-2589-4a73-a5a8-09aa3478c127-0',
            usage_metadata={'input_tokens': 30, 'output_tokens': 53, 'total_tokens': 83}
        ),
        'poem': AIMessage(
            content="Here is a short poem about mountains:\n\nGranite peaks touch the morning sky,\nNature's 
beauty, reaching for the eye.",
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:1b',
                'created_at': '2025-01-13T16:34:20.395705Z',
                'message': {'role': 'assistant', 'content': ''},
                'done_reason': 'stop',
                'done': True,
                'total_duration': 480513750,
                'load_duration': 32502125,
                'prompt_eval_count': 35,
                'prompt_eval_duration': 149000000,
                'eval_count': 26,
                'eval_duration': 297000000
            },
            id='run-fd81f0bc-f78f-42f2-84d1-2f41618e702e-0',
            usage_metadata={'input_tokens': 35, 'output_tokens': 26, 'total_tokens': 61}
        )
    }
]

CPU times: user 69.1 ms, sys: 7.15 ms, total: 76.3 ms
Wall time: 863 ms
